# Bibliotecas

In [1]:
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import funcoes_bert as fb

## Configurações

In [2]:
# force CPU (make CPU visible)
cpus = tf.config.experimental.list_physical_devices('CPU')
print(cpus)
tf.config.set_visible_devices([], 'GPU')  # hide the GPU
tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Constantes

In [3]:
SEED = 42

# Lendo os Arquivos

In [4]:
arquivos = ['hn_balanceado', 'hn_desbalanceado', 'sn_balanceado', 'sn_desbalanceado']

lista_df = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/{arquivo}.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df.append(df)

In [5]:
lista_df[0].dtypes

descricao_1    object
ean_1          object
titulo_1       object
url_1          object
titulo_cb_1    object
loja_1         object
descricao_2    object
ean_2          object
titulo_2       object
url_2          object
titulo_cb_2    object
loja_2         object
match           int64
dtype: object

In [6]:
print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {arquivos[0].shape[0]}\t| {arquivos[2].shape[0]}\t|\nDesbalanceado\t| {arquivos[1].shape[0]}\t| {arquivos[3].shape[0]}\t|")

(13290, 13)

In [7]:
df[df["titulo_1"]==df["titulo_2"]]

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2,match
2231,Esmaltec traz o fogão feito especialmente para...,7899081743783,Fogão 4 Bocas a Gás Esmaltec Esmeralda Glass 4...,https://www.amazon.com.br/Esmaltec-Esmeralda-A...,fogão 4 bocas a gás esmaltec esmeralda glass 4...,amazon,Esmaltec traz o fogão feito especialmente para...,7899081743783,Fogão 4 Bocas a Gás Esmaltec Esmeralda Glass 4...,https://www.americanas.com.br/produto/2599745729,fogão 4 bocas a gás esmaltec esmeralda glass 4...,americanas,1
7817,NaN,7892509119160,Smartphone Samsung Galaxy S20 Fe 128GB 4G Wi-F...,https://www.amazon.com.br/Smartphone-Samsung-G...,smartphone samsung galaxy s20 fe 128gb 4g wi-f...,amazon,Todas as informações divulgadas são de respons...,7892509119160,Smartphone Samsung Galaxy S20 Fe 128GB 4G Wi-F...,https://www.americanas.com.br/produto/3234381133,smartphone samsung galaxy s20 fe 128gb 4g wi-f...,americanas,1
10407,Todas as informações divulgadas são de respons...,7892509118439,Smartphone Samsung Galaxy A32 128GB 4G Wi-Fi T...,https://www.americanas.com.br/produto/2969968331,smartphone samsung galaxy a32 128gb 4g wi-fi t...,americanas,NaN,7892509118439,Smartphone Samsung Galaxy A32 128GB 4G Wi-Fi T...,https://www.amazon.com.br/Smartphone-Samsung-G...,smartphone samsung galaxy a32 128gb 4g wi-fi t...,amazon,1


# BERT

## Rodando pipeline experimento

In [19]:
lista_df_resultado = []
for nome, df in zip(arquivos, lista_df):
    
    X = df[["titulo_1", "titulo_2"]]#[df["titulo_1"].to_list(), df["titulo_2"].to_list()]
    y = df["match"].to_list()

    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2, random_state = SEED, stratify = y_train_valid)

    (nome, historico, y_test, y_pred) = fb.pipeline_bert(nome, X_train[:20], y_train[:20], X_valid[:20], y_valid[:20], X_test[:20], y_test[:20])

    pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/{nome}_historico.csv', index = False)

    report = classification_report(y_test, y_pred, output_dict = True)
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = nome

    df_resultado.to_csv(f'Dados/Resultados/{nome}_resultado.csv', index = True)

    lista_df_resultado.append(df_resultado)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 53s 53s/step - loss: 0.8070 - accuracy: 0.2000 - val_loss: 0.6715 - val_accuracy: 0.5500
Epoch 2/3
1/1 [==============================] - 33s 33s/step - loss: 0.6698 - accuracy: 0.5000 - val_loss: 0.5721 - val_accuracy: 0.9000
Epoch 3/3
1/1 [==============================] - 23s 23s/step - loss: 0.5707 - accuracy: 0.8500 - val_loss: 0.5059 - val_accuracy: 0.9000


c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
All PyTorch model weights were used when initializing T

Epoch 1/3
1/1 [==============================] - 55s 55s/step - loss: 0.6752 - accuracy: 0.6000 - val_loss: 0.5657 - val_accuracy: 0.8500
Epoch 2/3
1/1 [==============================] - 34s 34s/step - loss: 0.5512 - accuracy: 0.9000 - val_loss: 0.5076 - val_accuracy: 0.9000
Epoch 3/3
1/1 [==============================] - 34s 34s/step - loss: 0.4922 - accuracy: 0.8500 - val_loss: 0.4627 - val_accuracy: 0.9000


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 54s 54s/step - loss: 0.7854 - accuracy: 0.2500 - val_loss: 0.6698 - val_accuracy: 0.5500
Epoch 2/3
1/1 [==============================] - 37s 37s/step - loss: 0.6915 - accuracy: 0.4500 - val_loss: 0.6045 - val_accuracy: 0.8000
Epoch 3/3
1/1 [==============================] - 35s 35s/step - loss: 0.5943 - accuracy: 0.8500 - val_loss: 0.5569 - val_accuracy: 1.0000


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 50s 50s/step - loss: 0.8377 - accuracy: 0.4000 - val_loss: 0.7325 - val_accuracy: 0.5500
Epoch 2/3
1/1 [==============================] - 34s 34s/step - loss: 0.6766 - accuracy: 0.5500 - val_loss: 0.6117 - val_accuracy: 0.6500
Epoch 3/3
1/1 [==============================] - 34s 34s/step - loss: 0.5579 - accuracy: 0.8000 - val_loss: 0.5257 - val_accuracy: 0.8000


In [ ]:
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95        18
           1       0.00      0.00      0.00         2

    accuracy                           0.90        20
   macro avg       0.45      0.50      0.47        20
weighted avg       0.81      0.90      0.85        20

[[18  0]
 [ 2  0]]


c:\Users\llvs2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llvs2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llvs2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
lista_df_resultado

[              precision  recall  f1-score  support         modelo
 0                  0.90     1.0  0.947368     18.0  hn_balanceado
 1                  0.00     0.0  0.000000      2.0  hn_balanceado
 accuracy           0.90     0.9  0.900000      0.9  hn_balanceado
 macro avg          0.45     0.5  0.473684     20.0  hn_balanceado
 weighted avg       0.81     0.9  0.852632     20.0  hn_balanceado,
               precision  recall  f1-score  support            modelo
 0                  0.80     1.0  0.888889     16.0  hn_desbalanceado
 1                  0.00     0.0  0.000000      4.0  hn_desbalanceado
 accuracy           0.80     0.8  0.800000      0.8  hn_desbalanceado
 macro avg          0.40     0.5  0.444444     20.0  hn_desbalanceado
 weighted avg       0.64     0.8  0.711111     20.0  hn_desbalanceado,
               precision  recall  f1-score  support         modelo
 0                  0.90     1.0  0.947368     18.0  sn_balanceado
 1                  0.00     0.0  0.000000

# Resultados

### Carregando e Concatenando

Lê os arquivos com os resultados e depois concatena em um único dataframe

In [20]:
lista_df_resultados = []
for nome in arquivos:

    df = pd.read_csv(f"Dados/Resultados/{nome}_resultado.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_resultados.append(df)

df_resultados = pd.concat(lista_df_resultados, ignore_index = False)

In [22]:
df_resultados.rename(columns = {'Unnamed: 0':'info'}, inplace = True)
df_resultados

,info,precision,recall,f1-score,support,modelo
0,0,0.900000,1.000000,0.947368,18.00,hn_balanceado
1,1,0.000000,0.000000,0.000000,2.00,hn_balanceado
2,accuracy,0.900000,0.900000,0.900000,0.90,hn_balanceado
3,macro avg,0.450000,0.500000,0.473684,20.00,hn_balanceado
4,weighted avg,0.810000,0.900000,0.852632,20.00,hn_balanceado
0,0,0.789474,0.937500,0.857143,16.00,hn_desbalanceado
1,1,0.000000,0.000000,0.000000,4.00,hn_desbalanceado
2,accuracy,0.750000,0.750000,0.750000,0.75,hn_desbalanceado
3,macro avg,0.394737,0.468750,0.428571,20.00,hn_desbalanceado
4,weighted avg,0.631579,0.750000,0.685714,20.00,hn_desbalanceado


### Salvando

Salvando todos os resultados em um único arquivo

In [23]:
df_resultados.to_csv(f'Dados/Resultados/bert_resultado.csv', index = False)